# Рекомендация тарифов

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы, которые хочет рекомендовать своим клиентам заказчик. Предобработка данных уже проведена.

Нужно построить модель для задачи классификации со значением *accuracy* 0.75 и выше и проверить её на тестовой выборке. На основании модели клиентам будут предложены оптимальные тарифы.

## Откроем и изучим файл

In [4]:
#импорт библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [5]:
#открытие датафрейма
df = pd.read_csv('users_behavior.csv')

In [6]:
#изучение датафрема
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [7]:
#изучение датафрема
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Описание столбцов:  
сalls — количество звонков,  
minutes — суммарная длительность звонков в минутах,  
messages — количество sms-сообщений,  
mb_used — израсходованный интернет-трафик в Мб,  
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).  

Предобработка по заданию не требуется, но дробные значения для количества звонков и сообщений некорректно оставлять, переведем в целочисленный тип.

In [8]:
df[['calls', 'messages']] = df[['calls', 'messages']].astype(int)

## Разделим данные на выборки

In [9]:
# выделение признаков и обучающего признака
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [10]:
# разделение данных на три выборки: обучающую, валидационную и тестовую в соотношении 3:1:1

#обучающая выборка и выборка, которая станет тестовой и валидационной
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.4, random_state=12345, stratify=target)

# валидационная и тестовая выборки
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, test_size=0.5, random_state=12345, stratify=target_test)

## Исследуем модели

**Дерево решений**

In [11]:
best_model_dt = None
best_result_dt = 0
best_depth_dt = 0

for depth in range(1, 20):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_train, target_train)
    predictions_dt = model_dt.predict(features_valid)
    result = accuracy_score(predictions_dt, target_valid)
    if result > best_result_dt:
        best_model_dt = model_dt
        best_result_dt = result
        best_depth_dt = depth
        
print("Глубина дерева лучшей модели DT:", best_depth_dt)
print("Accuracy лучшей модели DT:", best_result_dt)

Глубина дерева лучшей модели DT: 8
Accuracy лучшей модели DT: 0.80248833592535


**Случайный лес**

In [12]:
best_model_rf = None
best_result_rf = 0
best_depth_rf = 0
best_est_rf = 0

for est in range(1, 51, 2):
    for depth in range(1, 15):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_rf.fit(features_train, target_train)
        result = model_rf.score(features_valid, target_valid)
        if result > best_result_rf:
            best_model_rf = model_rf
            best_result_rf = result
            best_depth_rf = depth
            best_est_rf = est
                   
print("Accuracy лучшей модели RF:", best_result_rf)
print("Глубина лучшей модели RF:", best_depth_rf)
print("Количество оценщиков:", best_est_rf)

Accuracy лучшей модели RF: 0.8227060653188181
Глубина лучшей модели RF: 9
Количество оценщиков: 39


Добавлю рандомное значение min_samples_leaf, а затем добавлю его в цикл, посмотрю, как изменяется accuracy

In [13]:
from tqdm import tqdm

best_model_rf = None
best_result_rf = 0
best_depth_rf = 0
best_est_rf = 0

for est in tqdm(range(1, 51, 2)):
    for depth in range(1, 15):
        model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth, min_samples_leaf=3)
        model_rf.fit(features_train, target_train)
        result = model_rf.score(features_valid, target_valid)
        if result > best_result_rf:
            best_model_rf = model_rf
            best_result_rf = result
            best_depth_rf = depth
            best_est_rf = est
                  
print("Accuracy лучшей модели RF:", best_result_rf)
print("Глубина лучшей модели RF:", best_depth_rf)
print("Количество оценщиков:", best_est_rf)

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:19<00:00,  1.26it/s]

Accuracy лучшей модели RF: 0.8195956454121306
Глубина лучшей модели RF: 9
Количество оценщиков: 43


**Логистическая регрессия**

In [14]:
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train)
result_lr = model_lr.score(features_valid, target_valid)

print("Accuracy лучшей модели LR:", result_lr)

Accuracy лучшей модели LR: 0.7387247278382582


Лучшая модель согласно метрике *accuracy*- RandomForestClassifier, проверим модель RFС с количеством оценщиков 39 и глубиной 9 на тестовой выборке.

## Проверка лучшей модели на тестовой выборке

In [15]:
model_rf = RandomForestClassifier(random_state=12345, n_estimators=39, max_depth=9)
model_rf.fit(features_train, target_train)
predictions_test_rf = model_rf.predict(features_test)
result_test_rf = accuracy_score(predictions_test_rf, target_test)

print("Accuracy RF на тестовой выборке:", result_test_rf)

Accuracy RF на тестовой выборке: 0.8118195956454122


На тестовой выборке точность модели составляет 0.812. Модель не переобучена, условие для значения точности модели - не менее 0.75 - соблюдено.


## Проверка модели на адекватность

In [16]:
from sklearn.dummy import DummyClassifier
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)

print("Accuracy DC", result_dc)

Accuracy DC 0.6936236391912908


Судя по точности модели DummyClassifier, выбранная модель RandomForestClassifier адекватна.

## Вывод:

В данном исследовании мы рассмотрели работу алгоритмов классификации для определения оптимальной модели, которая будет выбирать подходящий тариф. Мы выбрали для анализа следующие алгоритмы:

DecisionTreeClassifier
RandomForestClassifier
LogisticRegression

Метрика, по которой мы оценивали - accuracy.

В результате исследования для формирования модели был выбран алгоритм GradientBoostingClassifier(max_depth=9, n_estimators=39). Он показал значения метрики accuracy на test-выборке: 0.8118.

Адекватность модели проверена на модели DummyClassifier.
